# Ansible


Ansible is a configuration management software.

Connects via ssh or docker to a list of inventory machines and executes a series of tasks eventually grouped in playbooks.

At first, create an inventory file with all your hosts.

In [ ]:
cd /notebooks/exercise-00  

In [ ]:
cat inventory

Now check if you can ping the local host.

In [ ]:
# Check connections versus the local host in the "course" group

!ansible -i inventory -m ping course

In [ ]:
# Pinging all hosts gives some errors too, due to missing hosts or no ssh-key exchange
!ansible -i inventory -m ping all

## Further on inventories

 You can split your servers in many inventory files, like

 - staging
 
```
# staging inventory file
# run with
# ansible -i staging ...
 [ws]
 staging-ws-[0:3]
 
 [jboss]
 staging-boss-[0:6]
```
 
 - production
 
```
# production inventory file
# run with
# ansible -i production ...
 [ws]
 ws-[0:3]
 
 [jboss]
 boss-[0:6]
```
 
 

# Playbooks

To run a group of tasks with ansible, just:

  - create a playbook.yml
  - run ansible-playbook -i inventory playbook.yml
  

A playbook is a list of tasks in yml format, something like

```
#
# playbook.yml
#
- name: All public traffic is redirected via https
  uri:
    url: http://{{server_host}}/
    validate_certs: false
    follow_redirects: none
    status_code: 301
    
- name: This webapp  is served
  uri:
    url: https://{{server_host}}/webapp-1
    validate_certs: false
    status_code: 200
    HEADER_testflag: test

- name: The WS is serverd and requires authentication
  uri:
    url: https://{{server_host}}/rest/v1/method
    validate_certs: false
    status_code: 401

```

In this case, instead of making actual installation|setup tasks, we just created a testsuite validating our deployment. Now we must write another playbook which takes care of deployng the actual machine.



## Testing our course environment

We can write a playbook to test our course environment.

In [ ]:
!cat python-course-test.yml

In [ ]:
!ansible-playbook -i inventory python-course-test.yml 

As you can see something is missing: this playbook is not going to modify our machine but only test that everything is in place. 

See ```ansible-playbook --check ``` and ``` --diff ``` for further infos.

We can run a setup playbook, conventionally named site.yml.


In [ ]:
!cat site.yml

In [ ]:
!ansible-playbook -i inventory site.yml --limit=course  # in this case the --limit does not change anything ;)